In [16]:
from project.models import Project
from public_data.models import OcsgeDiff, CouvertureSol, Departement, Commune
from utils.db import cast_sum
from django.contrib.gis.db.models.functions import Centroid
from django.contrib.gis.db.models import Union
from django.db.models import Q

project = Project.objects.order_by("?").first()
project

project = Project.objects.get(pk=1470)
project

<Project: Diagnostic de Auch>

In [17]:
qs = project.cities.all().select_related("epci", "departement", "scot", "departement__region")
qs.count()

1

In [18]:
city_set = set()
epci = set()
scot = set()
departement = set()
region = set()
for city in qs:
    city_set.add(city)
    epci.add(city.epci)
    scot.add(city.scot)
    departement.add(city.departement)
    region.add(city.departement.region)
city_set, epci, scot, departement, region

({<Commune: Auch (32013)>},
 {<Epci: CA Grand Auch Coeur de Gascogne>},
 {<Scot: SCOT DE GASCOGNE>},
 {<Departement: Gers>},
 {<Region: Occitanie>})

In [15]:
next(iter(city_set))

<Commune: Ablain-Saint-Nazaire (62001)>